<img src="../static/logopython.png" alt="Logo Python" style="width: 300px; display: inline"/>
<img src="../static/deimoslogo.png" alt="Logo Deimos" style="width: 300px; display: inline"/>

# Clase 8: Ejercicios prácticos

En esta clase vamos a afianzar los conocimientos de Python que acabamos de adquirir haciendo algunos ejercicios sobre las herramientas del lenguaje Python para trabajar con concurrencia y paralelismo

## Ejercicio 1

La clase *Pool* del módulo *multiprocessing* tiene una función [*map*](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool.map). Utilizala para ejecutar una sencilla función que, aplicada sobre una lista de números, devuelva el número multiplicado por dos.

La operación ha de ejecutarse en un pool de 2 procesos paralelos

In [1]:
# Tu codigo aqui

import multiprocessing

def worker(num):
    return num*2

if __name__ == '__main__': 
    nums = [1,2,3,4,5,6,7,8,9] 
    p = multiprocessing.Pool(2)
    values = p.map(worker,nums) 
    
    p.close() 
    p.join()
    
    print(values) 

[2, 4, 6, 8, 10, 12, 14, 16, 18]


## Ejercicio 2

Vamos a crear nuestra propia clase que herede de [*Thread*](https://docs.python.org/3/library/threading.html?highlight=threading#threading.Thread), llamada *AsyncZip*. Servirá para crear un fichero zip a partir de un fichero cualquiera.

Completa el código a continuación

__Pista__: Puedes usar el módulo [*zipfile*](https://docs.python.org/3/library/zipfile.html)

__Nota__: Puedes mejorar el código añadiendo la posibilidad de introducir el fichero de entrada y el nombre del fichero de salida como argumentos de línea de comandos, pero no es un requisito

In [4]:
# Completa el código
import threading, zipfile

class AsyncZip(threading.Thread):
    def __init__(self, infile, outfile):
        threading.Thread.__init__(self)
        self.infile = infile
        self.outfile = outfile
        
    def run(self):
        f = zipfile.ZipFile(self.outfile, 'w', zipfile.ZIP_DEFLATED)
        f.write(self.infile)
        f.close()
        print('Finished background zip of {}'.format(self.infile))

background = AsyncZip('stockholm_td_adj.tsv', 'stockholm_td_adj.zip')
background.start()
print('Se va a crear un fichero zip en un hilo aparte.')

background.join() # Wait for the background task to finish
print('Hilo terminado.')

Se va a crear un fichero zip en un hilo aparte.
Finished background zip of stockholm_td_adj.tsv
Hilo terminado.


## Ejercicio 3

Implementar la corrutina *grep* en el siguiente trozo de código 

In [ ]:
def printer():
    """
        Corrutina que simplemente recibe datos y los imprime por pantalla
    """
    print("Preparado para imprimir...")
    try:
        while True:
            line = (yield)
            print(line)
    except GeneratorExit:
        print("Cerrando printer")

# Esta es la función a implementar. Tiene que:
# 1. Recibir datos del programa (g.send)
# 2. Filtrar esos datos y mandárselos a la corrutina que imprime por pantalla (consumer)
def grep(pattern, consumer):
    """
        Corrutina que filtra datos y se los manda a otra corrutina
    """
    print("Buscando la cadena {}".format(pattern))

    try:
        while True:
            s = (yield)
            if pattern in s:
                consumer.send(s)
    except GeneratorExit:
        print("Cerrando grep")
        consumer.close()


# Creamos la corrutina final y la arrancamos (las corrutinas se arrancan con send(None))
p = printer()
p.send(None)

# Creamos la corrutina que filtra y la arrancamos
g = grep("ending", p)
g.send(None)

# Este es el texto que metemos en nuestra cadena de corrutinas, de manera que:
# 1. La corrutina filtra las palabas
# 2. Le manda a la corrutina printer las palabras filtradas para que las imprima
text = 'Commending spending is offending to people pending lending!'

# Vamos mandando texto a la corrutina principal
for line in text.split():
    g.send(line)

#  Y cerramos
g.close()



## Ejercicio 4

Vamos a usar el módulo [aiohttp](http://aiohttp.readthedocs.io/) para obtener datos de urls mediante el uso de corrutinas. Este módulo básicamente proporciona corrutinas que pueden ser llamadas desde las nuestras, igual que en el ejemplo básico llamábamos a *asyncio.sleep*. 

Completar el método *fetch_page* que se muestra abajo para que muestre por pantalla el contenido de las llamadas a las urls (datos json). 

__Pista__: Posiblemente te resulten útiles estas dos corrutinas: [ClientSession.get](http://aiohttp.readthedocs.io/en/stable/client_reference.html#aiohttp.ClientSession.get), [ClientResponse,text](http://aiohttp.readthedocs.io/en/stable/client_reference.html#aiohttp.ClientResponse.text)



In [ ]:
import asyncio
import aiohttp
import json

# Completa este metodo
@asyncio.coroutine
def fetch_page(url):
    response = yield from aiohttp.request('GET', url)
    assert response.status == 200
    content = yield from response.read()
    print('URL: {0}:  Content: {1}'.format(url, json.loads(content.decode('utf-8'))))


def main():
    loop = asyncio.get_event_loop()
    tasks = [
        asyncio.ensure_future(fetch_page('http://jsonplaceholder.typicode.com/posts/1')),
        asyncio.ensure_future(fetch_page('http://jsonplaceholder.typicode.com/posts/2')),
        asyncio.ensure_future(fetch_page('http://jsonplaceholder.typicode.com/posts/3'))]
    loop.run_until_complete(asyncio.wait(tasks))
    loop.close()


if __name__ == "__main__":
    main()


## Ejercicio 5

En este ejercicio vamos a obtener los cinco posts más populares del día en 3 subredits de la web [reddit](https://www.reddit.com/). Usaremos corrutinas y la nueva sintáxis async/await, disponible desde Python 3.5. Recuerda, por tanto, que:

* No será necesario el decorador @asyncio.coroutine. En su lugar debes declarar las corrutinas como async
* En lugar de *yield from* vas a usar *await*

Completa el código a continuación. Fíjate en el ejemplo anterior para ver cómo se usa la api de aiohttp. En cuanto a la información a mostrar por pantalla, estudia la respuesta JSON que te devuelve cada una de las llamadas para saber qué campos necesitas. En pantalla, para cada url, has de mostrar:

* La puntuación del post (el score)
* El título del mismo
* El enlace al post

Es suficiente con que lo muestre en modo texto, no hay que hacer ninguna floritura gráfica

In [ ]:
import asyncio
import aiohttp
import json

async def get_json(client, url):
    async with client.get(url) as response:
        assert response.status == 200
        return await response.read()

async def get_reddit_top(subreddit, client):
    data1 = await get_json(client, 'https://www.reddit.com/r/' + subreddit + '/top.json?sort=top&t=day&limit=5')

    j = json.loads(data1.decode('utf-8'))
    for i in j['data']['children']:
        score = i['data']['score']
        title = i['data']['title']
        link = i['data']['url']
        print(str(score) + ': ' + title + ' (' + link + ')')

    print('DONE:', subreddit + '\n')


def main():
    loop = asyncio.get_event_loop()
    client = aiohttp.ClientSession(loop=loop)


    tasks = [
        asyncio.ensure_future(get_reddit_top('python', client)),
        asyncio.ensure_future(get_reddit_top('programming', client)),
        asyncio.ensure_future(get_reddit_top('compsci', client))
    ]

    loop.run_until_complete(asyncio.wait(tasks))

    loop.stop()
    client.close()


if __name__ == "__main__":
    main()


##### <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es"><img alt="Licencia Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Curso Python</span> por <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Jorge Arévalo</span> se distribuye bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es">Licencia Creative Commons Atribución 4.0 Internacional</a>.

---
_Las siguientes celdas contienen configuración del Notebook_

_Para visualizar y utlizar los enlaces a Twitter el notebook debe ejecutarse como [seguro](http://ipython.org/ipython-doc/dev/notebook/security.html)_

    File > Trusted Notebook

In [1]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_file = '../static/styles/style.css'
HTML(open(css_file, "r").read())